In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
from config import census_api_key, mysql_pw

# Extraction and transformation of public health statistics for 500 US cities

**Source:** Kaggle (https://www.kaggle.com/cdc/500-cities)

**Description:** This data was collected by Centers for Disease Control and Prevention, National Center for Chronic Disease Prevention and Health Promotion, Division of Population Health (2016)

**Data collection and preparation credit:** David Chen

In [2]:
cdc_file = "data/500_Cities_CDC.csv"
cdc_data_df = pd.read_csv(cdc_file)
cdc_data_df.head()

,StateAbbr,PlaceName,PlaceFIPS,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ACCESS2_AdjPrev,ACCESS2_Adj95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,SLEEP_Adj95CI,STROKE_CrudePrev,STROKE_Crude95CI,STROKE_AdjPrev,STROKE_Adj95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,TEETHLOST_AdjPrev,TEETHLOST_Adj95CI,Geolocation
0,AL,Birmingham,107000,212237,22.6,"(22.1, 23.0)",21.4,"(21.0, 21.8)",32.6,"(32.5, 32.8)",...,"(46.6, 47.0)",5.0,"( 5.0, 5.1)",5.0,"( 5.0, 5.1)",26.1,"(25.1, 27.2)",25.9,"(25.0, 26.9)","(33.52756637730, -86.7988174678)"
1,AL,Hoover,135896,81619,10.6,"(10.2, 11.1)",10.2,"( 9.7, 10.7)",26.3,"(26.0, 26.6)",...,"(34.2, 35.0)",2.3,"( 2.2, 2.3)",2.2,"( 2.1, 2.3)",9.6,"( 8.6, 10.8)",9.5,"( 8.5, 10.9)","(33.37676027290, -86.8051937568)"
2,AL,Huntsville,137000,180105,17.4,"(16.9, 17.8)",16.3,"(15.9, 16.7)",30.0,"(29.8, 30.2)",...,"(39.4, 40.0)",3.3,"( 3.3, 3.4)",3.2,"( 3.1, 3.2)",14.9,"(14.1, 15.7)",14.7,"(13.8, 15.5)","(34.69896926710, -86.6387042882)"
3,AL,Mobile,150000,195111,20.0,"(19.6, 20.4)",19.1,"(18.7, 19.5)",33.1,"(32.9, 33.2)",...,"(42.0, 42.4)",4.3,"( 4.3, 4.4)",4.1,"( 4.0, 4.1)",24.3,"(23.4, 25.3)",24.1,"(23.1, 25.0)","(30.67762486480, -88.1184482714)"
4,AL,Montgomery,151000,205764,19.7,"(19.2, 20.2)",18.5,"(18.1, 19.0)",31.0,"(30.8, 31.2)",...,"(41.0, 41.5)",4.0,"( 3.9, 4.1)",4.1,"( 4.0, 4.1)",21.2,"(20.3, 22.2)",21.2,"(20.1, 22.2)","(32.34726453330, -86.2677059552)"


In [6]:
cdc_data_df.columns.tolist()

['StateAbbr',
 'PlaceName',
 'PlaceFIPS',
 'Population2010',
 'ACCESS2_CrudePrev',
 'ACCESS2_Crude95CI',
 'ACCESS2_AdjPrev',
 'ACCESS2_Adj95CI',
 'ARTHRITIS_CrudePrev',
 'ARTHRITIS_Crude95CI',
 'ARTHRITIS_AdjPrev',
 'ARTHRITIS_Adj95CI',
 'BINGE_CrudePrev',
 'BINGE_Crude95CI',
 'BINGE_AdjPrev',
 'BINGE_Adj95CI',
 'BPHIGH_CrudePrev',
 'BPHIGH_Crude95CI',
 'BPHIGH_AdjPrev',
 'BPHIGH_Adj95CI',
 'BPMED_CrudePrev',
 'BPMED_Crude95CI',
 'BPMED_AdjPrev',
 'BPMED_Adj95CI',
 'CANCER_CrudePrev',
 'CANCER_Crude95CI',
 'CANCER_AdjPrev',
 'CANCER_Adj95CI',
 'CASTHMA_CrudePrev',
 'CASTHMA_Crude95CI',
 'CASTHMA_AdjPrev',
 'CASTHMA_Adj95CI',
 'CHD_CrudePrev',
 'CHD_Crude95CI',
 'CHD_AdjPrev',
 'CHD_Adj95CI',
 'CHECKUP_CrudePrev',
 'CHECKUP_Crude95CI',
 'CHECKUP_AdjPrev',
 'CHECKUP_Adj95CI',
 'CHOLSCREEN_CrudePrev',
 'CHOLSCREEN_Crude95CI',
 'CHOLSCREEN_AdjPrev',
 'CHOLSCREEN_Adj95CI',
 'COLON_SCREEN_CrudePrev',
 'COLON_SCREEN_Crude95CI',
 'COLON_SCREEN_AdjPrev',
 'COLON_SCREEN_Adj95CI',
 'COPD_CrudePre

In [4]:
column_list = ['ACCESS2_CrudePrev',
 'ACCESS2_Crude95CI',
 'ACCESS2_AdjPrev',
 'ACCESS2_Adj95CI',
 'ARTHRITIS_CrudePrev',
 'ARTHRITIS_Crude95CI',
 'ARTHRITIS_AdjPrev',
 'ARTHRITIS_Adj95CI',
 'BINGE_CrudePrev',
 'BINGE_Crude95CI',
 'BINGE_AdjPrev',
 'BINGE_Adj95CI',
 'BPHIGH_CrudePrev',
 'BPHIGH_Crude95CI',
 'BPHIGH_AdjPrev',
 'BPHIGH_Adj95CI',
 'BPMED_CrudePrev',
 'BPMED_Crude95CI',
 'BPMED_AdjPrev',
 'BPMED_Adj95CI',
 'CANCER_CrudePrev',
 'CANCER_Crude95CI',
 'CANCER_AdjPrev',
 'CANCER_Adj95CI',
 'CASTHMA_CrudePrev',
 'CASTHMA_Crude95CI',
 'CASTHMA_AdjPrev',
 'CASTHMA_Adj95CI',
 'CHD_CrudePrev',
 'CHD_Crude95CI',
 'CHD_AdjPrev',
 'CHD_Adj95CI',
 'CHECKUP_CrudePrev',
 'CHECKUP_Crude95CI',
 'CHECKUP_AdjPrev',
 'CHECKUP_Adj95CI',
 'CHOLSCREEN_CrudePrev',
 'CHOLSCREEN_Crude95CI',
 'CHOLSCREEN_AdjPrev',
 'CHOLSCREEN_Adj95CI',
 'COLON_SCREEN_CrudePrev',
 'COLON_SCREEN_Crude95CI',
 'COLON_SCREEN_AdjPrev',
 'COLON_SCREEN_Adj95CI',
 'COPD_CrudePrev',
 'COPD_Crude95CI',
 'COPD_AdjPrev',
 'COPD_Adj95CI',
 'COREM_CrudePrev',
 'COREM_Crude95CI',
 'COREM_AdjPrev',
 'COREM_Adj95CI',
 'COREW_CrudePrev',
 'COREW_Crude95CI',
 'COREW_AdjPrev',
 'COREW_Adj95CI',
 'CSMOKING_CrudePrev',
 'CSMOKING_Crude95CI',
 'CSMOKING_AdjPrev',
 'CSMOKING_Adj95CI',
 'DENTAL_CrudePrev',
 'DENTAL_Crude95CI',
 'DENTAL_AdjPrev',
 'DENTAL_Adj95CI',
 'DIABETES_CrudePrev',
 'DIABETES_Crude95CI',
 'DIABETES_AdjPrev',
 'DIABETES_Adj95CI',
 'HIGHCHOL_CrudePrev',
 'HIGHCHOL_Crude95CI',
 'HIGHCHOL_AdjPrev',
 'HIGHCHOL_Adj95CI',
 'KIDNEY_CrudePrev',
 'KIDNEY_Crude95CI',
 'KIDNEY_AdjPrev',
 'KIDNEY_Adj95CI',
 'LPA_CrudePrev',
 'LPA_Crude95CI',
 'LPA_AdjPrev',
 'LPA_Adj95CI',
 'MAMMOUSE_CrudePrev',
 'MAMMOUSE_Crude95CI',
 'MAMMOUSE_AdjPrev',
 'MAMMOUSE_Adj95CI',
 'MHLTH_CrudePrev',
 'MHLTH_Crude95CI',
 'MHLTH_AdjPrev',
 'MHLTH_Adj95CI',
 'OBESITY_CrudePrev',
 'OBESITY_Crude95CI',
 'OBESITY_AdjPrev',
 'OBESITY_Adj95CI',
 'PAPTEST_CrudePrev',
 'PAPTEST_Crude95CI',
 'PAPTEST_AdjPrev',
 'PAPTEST_Adj95CI',
 'PHLTH_CrudePrev',
 'PHLTH_Crude95CI',
 'PHLTH_AdjPrev',
 'PHLTH_Adj95CI',
 'SLEEP_CrudePrev',
 'SLEEP_Crude95CI',
 'SLEEP_AdjPrev',
 'SLEEP_Adj95CI',
 'STROKE_CrudePrev',
 'STROKE_Crude95CI',
 'STROKE_AdjPrev',
 'STROKE_Adj95CI',
 'TEETHLOST_CrudePrev',
 'TEETHLOST_Crude95CI',
 'TEETHLOST_AdjPrev',
 'TEETHLOST_Adj95CI']
print(column_list)

['ACCESS2_CrudePrev', 'ACCESS2_Crude95CI', 'ACCESS2_AdjPrev', 'ACCESS2_Adj95CI', 'ARTHRITIS_CrudePrev', 'ARTHRITIS_Crude95CI', 'ARTHRITIS_AdjPrev', 'ARTHRITIS_Adj95CI', 'BINGE_CrudePrev', 'BINGE_Crude95CI', 'BINGE_AdjPrev', 'BINGE_Adj95CI', 'BPHIGH_CrudePrev', 'BPHIGH_Crude95CI', 'BPHIGH_AdjPrev', 'BPHIGH_Adj95CI', 'BPMED_CrudePrev', 'BPMED_Crude95CI', 'BPMED_AdjPrev', 'BPMED_Adj95CI', 'CANCER_CrudePrev', 'CANCER_Crude95CI', 'CANCER_AdjPrev', 'CANCER_Adj95CI', 'CASTHMA_CrudePrev', 'CASTHMA_Crude95CI', 'CASTHMA_AdjPrev', 'CASTHMA_Adj95CI', 'CHD_CrudePrev', 'CHD_Crude95CI', 'CHD_AdjPrev', 'CHD_Adj95CI', 'CHECKUP_CrudePrev', 'CHECKUP_Crude95CI', 'CHECKUP_AdjPrev', 'CHECKUP_Adj95CI', 'CHOLSCREEN_CrudePrev', 'CHOLSCREEN_Crude95CI', 'CHOLSCREEN_AdjPrev', 'CHOLSCREEN_Adj95CI', 'COLON_SCREEN_CrudePrev', 'COLON_SCREEN_Crude95CI', 'COLON_SCREEN_AdjPrev', 'COLON_SCREEN_Adj95CI', 'COPD_CrudePrev', 'COPD_Crude95CI', 'COPD_AdjPrev', 'COPD_Adj95CI', 'COREM_CrudePrev', 'COREM_Crude95CI', 'COREM_AdjPre

In [5]:
len(column_list)

112

In [7]:
for x in range (len(column_list)):
    if x % 4 == 2:
        print (column_list[x])

ACCESS2_AdjPrev
ARTHRITIS_AdjPrev
BINGE_AdjPrev
BPHIGH_AdjPrev
BPMED_AdjPrev
CANCER_AdjPrev
CASTHMA_AdjPrev
CHD_AdjPrev
CHECKUP_AdjPrev
CHOLSCREEN_AdjPrev
COLON_SCREEN_AdjPrev
COPD_AdjPrev
COREM_AdjPrev
COREW_AdjPrev
CSMOKING_AdjPrev
DENTAL_AdjPrev
DIABETES_AdjPrev
HIGHCHOL_AdjPrev
KIDNEY_AdjPrev
LPA_AdjPrev
MAMMOUSE_AdjPrev
MHLTH_AdjPrev
OBESITY_AdjPrev
PAPTEST_AdjPrev
PHLTH_AdjPrev
SLEEP_AdjPrev
STROKE_AdjPrev
TEETHLOST_AdjPrev


In [8]:
new_cdc_data_df = cdc_data_df[["StateAbbr",
"PlaceName",
"PlaceFIPS",
"ACCESS2_AdjPrev",
"ARTHRITIS_AdjPrev",
"BINGE_AdjPrev",
"BPHIGH_AdjPrev",
"BPMED_AdjPrev",
"CANCER_AdjPrev",
"CASTHMA_AdjPrev",
"CHD_AdjPrev",
"CHECKUP_AdjPrev",
"CHOLSCREEN_AdjPrev",
"COLON_SCREEN_AdjPrev",
"COPD_AdjPrev",
"COREM_AdjPrev",
"COREW_AdjPrev",
"CSMOKING_AdjPrev",
"DENTAL_AdjPrev",
"DIABETES_AdjPrev",
"HIGHCHOL_AdjPrev",
"KIDNEY_AdjPrev",
"LPA_AdjPrev",
"MAMMOUSE_AdjPrev",
"MHLTH_AdjPrev",
"OBESITY_AdjPrev",
"PAPTEST_AdjPrev",
"PHLTH_AdjPrev",
"SLEEP_AdjPrev",
"STROKE_AdjPrev",
"TEETHLOST_AdjPrev"]].copy()
new_cdc_data_df.head()

,StateAbbr,PlaceName,PlaceFIPS,ACCESS2_AdjPrev,ARTHRITIS_AdjPrev,BINGE_AdjPrev,BPHIGH_AdjPrev,BPMED_AdjPrev,CANCER_AdjPrev,CASTHMA_AdjPrev,...,KIDNEY_AdjPrev,LPA_AdjPrev,MAMMOUSE_AdjPrev,MHLTH_AdjPrev,OBESITY_AdjPrev,PAPTEST_AdjPrev,PHLTH_AdjPrev,SLEEP_AdjPrev,STROKE_AdjPrev,TEETHLOST_AdjPrev
0,AL,Birmingham,107000,21.4,32.6,11.4,45.9,70.5,6.1,11.4,...,3.3,31.7,79.7,17.0,39.0,80.1,18.3,46.9,5.0,25.9
1,AL,Hoover,135896,10.2,25.3,15.7,31.9,63.8,6.8,8.2,...,2.2,17.8,82.1,10.8,24.7,86.3,10.0,34.6,2.2,9.5
2,AL,Huntsville,137000,16.3,29.4,12.3,36.7,66.9,6.5,9.6,...,2.6,24.9,80.0,14.0,32.0,83.5,13.9,39.7,3.2,14.7
3,AL,Mobile,150000,19.1,31.8,12.8,43.0,69.3,6.5,10.7,...,3.1,27.4,80.1,15.8,37.6,81.9,16.3,42.2,4.1,24.1
4,AL,Montgomery,151000,18.5,31.4,12.2,40.6,70.3,6.4,10.8,...,3.1,27.9,80.2,15.5,36.8,83.0,16.2,41.3,4.1,21.2


In [9]:
new_cdc_data_df_1 = new_cdc_data_df.rename(columns={"StateAbbr": "State",
"PlaceName": "City",
"PlaceFIPS": "PlaceFIPS",
"ACCESS2_AdjPrev": "ACCESS2",
"ARTHRITIS_AdjPrev" :"ARTHRITIS",
"BINGE_AdjPrev" :"BINGE",
"BPHIGH_AdjPrev" :"BPHIGH",
"BPMED_AdjPrev" :"BPMED",
"CANCER_AdjPrev" :"CANCER",
"CASTHMA_AdjPrev" :"CASTHMA",
"CHD_AdjPrev" :"CHD",
"CHECKUP_AdjPrev" :"CHECKUP",
"CHOLSCREEN_AdjPrev" :"CHOLSCREEN",
"COLON_SCREEN_AdjPrev" :"COLON_SCREEN",
"COPD_AdjPrev" :"COPD",
"COREM_AdjPrev" :"COREM",
"COREW_AdjPrev" :"COREW",
"CSMOKING_AdjPrev" :"CSMOKING",
"DENTAL_AdjPrev" :"DENTAL",
"DIABETES_AdjPrev" :"DIABETES",
"HIGHCHOL_AdjPrev" :"HIGHCHOL",
"KIDNEY_AdjPrev" :"KIDNEY",
"LPA_AdjPrev" :"LPA",
"MAMMOUSE_AdjPrev" :"MAMMOUSE",
"MHLTH_AdjPrev" :"MHLTH",
"OBESITY_AdjPrev" :"OBESITY",
"PAPTEST_AdjPrev" :"PAPTEST",
"PHLTH_AdjPrev" :"PHLTH",
"SLEEP_AdjPrev" :"SLEEP",
"STROKE_AdjPrev" :"STROKE",
"TEETHLOST_AdjPrev" :"TEETHLOST"    
})
new_cdc_data_df_1.head()

,State,City,PlaceFIPS,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,...,KIDNEY,LPA,MAMMOUSE,MHLTH,OBESITY,PAPTEST,PHLTH,SLEEP,STROKE,TEETHLOST
0,AL,Birmingham,107000,21.4,32.6,11.4,45.9,70.5,6.1,11.4,...,3.3,31.7,79.7,17.0,39.0,80.1,18.3,46.9,5.0,25.9
1,AL,Hoover,135896,10.2,25.3,15.7,31.9,63.8,6.8,8.2,...,2.2,17.8,82.1,10.8,24.7,86.3,10.0,34.6,2.2,9.5
2,AL,Huntsville,137000,16.3,29.4,12.3,36.7,66.9,6.5,9.6,...,2.6,24.9,80.0,14.0,32.0,83.5,13.9,39.7,3.2,14.7
3,AL,Mobile,150000,19.1,31.8,12.8,43.0,69.3,6.5,10.7,...,3.1,27.4,80.1,15.8,37.6,81.9,16.3,42.2,4.1,24.1
4,AL,Montgomery,151000,18.5,31.4,12.2,40.6,70.3,6.4,10.8,...,3.1,27.9,80.2,15.5,36.8,83.0,16.2,41.3,4.1,21.2


In [10]:
place_fips = new_cdc_data_df_1["PlaceFIPS"].tolist()
print (place_fips)

[107000, 135896, 137000, 150000, 151000, 177256, 203000, 404720, 412000, 427400, 427820, 446000, 454050, 455000, 465000, 471510, 473000, 477000, 485540, 523290, 524550, 535710, 541000, 566080, 600562, 600884, 602000, 602252, 602364, 603526, 603666, 604982, 606000, 608786, 608954, 611194, 611530, 613014, 613210, 613214, 613392, 613588, 614218, 615044, 616000, 616350, 616532, 617918, 619766, 621712, 622020, 622230, 622804, 623182, 624638, 624680, 626000, 627000, 628000, 629000, 630000, 632548, 633000, 633182, 633434, 636000, 636448, 636546, 636770, 639496, 639892, 640130, 641992, 643000, 644000, 644574, 645484, 646842, 646898, 647766, 648256, 648354, 649270, 649670, 650076, 650258, 651182, 652526, 653000, 653322, 653896, 653980, 654652, 655156, 656000, 656700, 657792, 658072, 659451, 659920, 659962, 660018, 660102, 660466, 660620, 662000, 662938, 664000, 664224, 665000, 665042, 666000, 667000, 668000, 668084, 668196, 668252, 668378, 669000, 669070, 669084, 669088, 669196, 670000, 670098,

In [11]:
for y in place_fips:
    print (str(y)[-5:])

07000
35896
37000
50000
51000
77256
03000
04720
12000
27400
27820
46000
54050
55000
65000
71510
73000
77000
85540
23290
24550
35710
41000
66080
00562
00884
02000
02252
02364
03526
03666
04982
06000
08786
08954
11194
11530
13014
13210
13214
13392
13588
14218
15044
16000
16350
16532
17918
19766
21712
22020
22230
22804
23182
24638
24680
26000
27000
28000
29000
30000
32548
33000
33182
33434
36000
36448
36546
36770
39496
39892
40130
41992
43000
44000
44574
45484
46842
46898
47766
48256
48354
49270
49670
50076
50258
51182
52526
53000
53322
53896
53980
54652
55156
56000
56700
57792
58072
59451
59920
59962
60018
60102
60466
60620
62000
62938
64000
64224
65000
65042
66000
67000
68000
68084
68196
68252
68378
69000
69070
69084
69088
69196
70000
70098
72016
73080
75000
77000
78120
78582
80000
80238
80812
80854
81204
81344
81554
81666
82590
82954
82996
84200
84550
85292
03455
04000
07850
12815
16000
20000
27425
32155
43000
45970
46465
62000
77290
83835
08000
18430
37000
50370
52000
55990
73000
8000

In [12]:
new_cdc_data_df_1.to_csv("data/500_Cities_CDC_Disease_Columns.csv", encoding='utf-8',index= False)

In [13]:
cdc_file_update_column = "data/500_Cities_CDC_Disease_Columns.csv"
cdc_data_df_update_column = pd.read_csv(cdc_file_update_column)
cdc_data_df_update_column.head()

,State,City,PlaceFIPS,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,...,KIDNEY,LPA,MAMMOUSE,MHLTH,OBESITY,PAPTEST,PHLTH,SLEEP,STROKE,TEETHLOST
0,AL,Birmingham,107000,21.4,32.6,11.4,45.9,70.5,6.1,11.4,...,3.3,31.7,79.7,17.0,39.0,80.1,18.3,46.9,5.0,25.9
1,AL,Hoover,135896,10.2,25.3,15.7,31.9,63.8,6.8,8.2,...,2.2,17.8,82.1,10.8,24.7,86.3,10.0,34.6,2.2,9.5
2,AL,Huntsville,137000,16.3,29.4,12.3,36.7,66.9,6.5,9.6,...,2.6,24.9,80.0,14.0,32.0,83.5,13.9,39.7,3.2,14.7
3,AL,Mobile,150000,19.1,31.8,12.8,43.0,69.3,6.5,10.7,...,3.1,27.4,80.1,15.8,37.6,81.9,16.3,42.2,4.1,24.1
4,AL,Montgomery,151000,18.5,31.4,12.2,40.6,70.3,6.4,10.8,...,3.1,27.9,80.2,15.5,36.8,83.0,16.2,41.3,4.1,21.2


# Extraction and Transformation of Census Data

**Source:** Census API (https://api.census.gov)

**Description:** This data was called by county FIPS for all

**Census data collection and preparation:** Steve Bogdan

In [14]:
# url for US Census data api call with codes for desired data fields

url = "https://api.census.gov/data/2017/acs/acs5?get=B01003_001E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B01001I_001E,B09020_001E,B15003_002E,B15003_017E,B15003_022E,B15003_023E,B15003_025E,B17001_002E,B27001_002E,B27001_030E,NAME&for=place:*&key=" + census_api_key

# Make api call and get json object of returned data 
response = requests.get(url)
response_json = response.json()

#Convert json object and make first row column names
census_df = pd.DataFrame(response_json)

census_df.columns = census_df.iloc[0]
census_df = census_df[1:]

census_df.head()

,B01003_001E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,...,B15003_017E,B15003_022E,B15003_023E,B15003_025E,B17001_002E,B27001_002E,B27001_030E,NAME,state,place
1,174,0,0,0,0,0,0,0,0,0,...,0,0,0,0,45,69,105,"Abanda CDP, Alabama",01,00100
2,2594,51,40,35,60,40,29,8,27,109,...,567,112,112,4,500,1248,1164,"Abbeville city, Alabama",01,00124
3,4404,49,67,84,46,67,0,25,65,201,...,755,283,103,38,701,2063,2326,"Adamsville city, Alabama",01,00460
4,725,15,12,38,13,5,0,24,14,29,...,171,59,38,0,246,385,340,"Addison town, Alabama",01,00484
5,318,0,5,2,0,1,6,2,2,14,...,52,5,1,0,155,139,179,"Akron town, Alabama",01,00676


In [16]:
# Dictionary to change dataframe column names from census code to meaningful names
column_names = {"B01003_001E":"total_pop", "B01001_020E":"male_65_66", "B01001_021E":"male_67_69",
                "B01001_022E":"male_70_74", "B01001_023E":"male_75_79", "B01001_024E":"male_80_84",
                "B01001_025E":"male_over_85", "B01001_044E":"female_65_66", "B01001_045E":"female_67_69",
                "B01001_046E":"female_70_74", "B01001_047E":"female_75_79", "B01001_048E":"female_80_84",
                "B01001_049E":"female_over_85", "B02001_002E":"white_pop", "B02001_003E":"black_pop",
                "B02001_004E":"native_amer_pop", "B02001_005E":"asian_pop", "B02001_006E":"pac_island_pop",
                "B02001_007E":"other_race_pop", "B01001I_001E":"hispanic_pop", "B15003_002E":"no_high_school",
                "B15003_017E":"high_school_grad", "B15003_022E":"bachelor_deg", "B15003_023E":"master_deg", 
                "B15003_025E":"doctorate_deg", "B17001_002E":"below_poverty", "B27001_002E":"male_w_health_ins",
                "B27001_030E":"female_w_health_ins", "place":"city_FIPS"}

census_df.rename(columns=column_names, 
                 inplace=True)

# Convert all columns with numeric values to numneric in the dataframe
for column in census_df.iloc[:, 0:29]:
    census_df[column] = pd.to_numeric(census_df[column])

# Combine health insurance data for male and female into one column
census_df["with_health_ins"] = census_df["male_w_health_ins"] + census_df["female_w_health_ins"]

# Combine city and state FIPS codes into one column to match format of the health data set
census_df["city_FIPS"] = census_df["state"] + census_df["city_FIPS"]

# Combine all age range columns into single column to get single value of population over 65
census_df["pop_over_65"] = census_df.iloc[:,1:13].sum(axis=1)

# Combine all columns of number of people with different types of degrees into single column of number with degrees
census_df["with_degree"] = census_df.iloc[:, 23:26].sum(axis=1)

# Select desired columns for final data set
census_demographics_df=census_df[["total_pop", "with_health_ins", "pop_over_65", "white_pop", "black_pop", "native_amer_pop",
                                "asian_pop", "pac_island_pop", "other_race_pop", "hispanic_pop", "no_high_school",
                                "high_school_grad", "with_degree", "below_poverty", "city_FIPS"]]

# Divide all columns with population numbers by the total population to convert to percentage
for column in census_demographics_df.iloc[:, 1:14]:
    census_demographics_df[column] = census_demographics_df[column]/census_demographics_df.iloc[:,0]

census_demographics_df.head()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,total_pop,with_health_ins,pop_over_65,white_pop,black_pop,native_amer_pop,asian_pop,pac_island_pop,other_race_pop,hispanic_pop,no_high_school,high_school_grad,with_degree,below_poverty,city_FIPS
1,174,1.000000,0.000000,0.862069,0.137931,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.258621,0100100
2,2594,0.929838,0.246723,0.563994,0.417502,0.000000,0.0,0.0,0.005397,0.045104,0.013107,0.218581,0.087895,0.192753,0100124
3,4404,0.996594,0.184832,0.471617,0.518619,0.000000,0.0,0.0,0.001589,0.001589,0.005223,0.171435,0.096276,0.159173,0100460
4,725,1.000000,0.240000,0.944828,0.000000,0.004138,0.0,0.0,0.000000,0.000000,0.006897,0.235862,0.133793,0.339310,0100484
5,318,1.000000,0.106918,0.220126,0.779874,0.000000,0.0,0.0,0.000000,0.000000,0.003145,0.163522,0.018868,0.487421,0100676


In [23]:
# Save the Census demographics data to .csv file
census_demographics_df.to_csv('data/city_health_demographics.csv', encoding="utf-8", header=1)

# Database creation and data upload

We use MySQL database to store and aggregate the data

**Database creation:** Brian E Reyes, Olesya Bondarenko

In [24]:
# Load census data from csv file into dataframe
census_final_df = pd.read_csv('data/city_health_demographics.csv', encoding="utf-8")
census_final_df.head()

,Unnamed: 0,total_pop,with_health_ins,pop_over_65,white_pop,black_pop,native_amer_pop,asian_pop,pac_island_pop,other_race_pop,hispanic_pop,no_high_school,high_school_grad,with_degree,below_poverty,city_FIPS
0,1,174,1.000000,0.000000,0.862069,0.137931,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.258621,100100
1,2,2594,0.929838,0.246723,0.563994,0.417502,0.000000,0.0,0.0,0.005397,0.045104,0.013107,0.218581,0.087895,0.192753,100124
2,3,4404,0.996594,0.184832,0.471617,0.518619,0.000000,0.0,0.0,0.001589,0.001589,0.005223,0.171435,0.096276,0.159173,100460
3,4,725,1.000000,0.240000,0.944828,0.000000,0.004138,0.0,0.0,0.000000,0.000000,0.006897,0.235862,0.133793,0.339310,100484
4,5,318,1.000000,0.106918,0.220126,0.779874,0.000000,0.0,0.0,0.000000,0.000000,0.003145,0.163522,0.018868,0.487421,100676


In [25]:
# Load health data from csv file into dataframe
health_final_df = pd.read_csv('data/500_Cities_CDC_Disease_Columns.csv', encoding="utf-8")
health_final_df.head()

,State,City,PlaceFIPS,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,...,KIDNEY,LPA,MAMMOUSE,MHLTH,OBESITY,PAPTEST,PHLTH,SLEEP,STROKE,TEETHLOST
0,AL,Birmingham,107000,21.4,32.6,11.4,45.9,70.5,6.1,11.4,...,3.3,31.7,79.7,17.0,39.0,80.1,18.3,46.9,5.0,25.9
1,AL,Hoover,135896,10.2,25.3,15.7,31.9,63.8,6.8,8.2,...,2.2,17.8,82.1,10.8,24.7,86.3,10.0,34.6,2.2,9.5
2,AL,Huntsville,137000,16.3,29.4,12.3,36.7,66.9,6.5,9.6,...,2.6,24.9,80.0,14.0,32.0,83.5,13.9,39.7,3.2,14.7
3,AL,Mobile,150000,19.1,31.8,12.8,43.0,69.3,6.5,10.7,...,3.1,27.4,80.1,15.8,37.6,81.9,16.3,42.2,4.1,24.1
4,AL,Montgomery,151000,18.5,31.4,12.2,40.6,70.3,6.4,10.8,...,3.1,27.9,80.2,15.5,36.8,83.0,16.2,41.3,4.1,21.2


In [26]:
# Connect to MySQL
connection_string = f'root:{mysql_pw}@localhost/'
engine = create_engine(f'mysql://{connection_string}')
connection = engine.connect()

In [30]:
# Create a new empty database
connection.execute('create schema city_health_demographics;')

In [31]:
# Create a connection to the new database
db_connection_string = f"root:{mysql_pw}@localhost/City_Health_Demographics"
db_engine = create_engine(f'mysql://{db_connection_string}')

In [32]:
# Push the transformed CDC 500 US city data to the 'city_health_demographics' database. 
# It will be stored as a new table called 'city_health'
health_final_df.to_sql(name='City_Health', con=db_engine, if_exists='append', index=False)

C:\Anaconda\lib\site-packages\pandas\io\sql.py:1144: UserWarning: The provided table name 'City_Health' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [33]:
# Push the transformed Census data to the 'city_health_demographics' database. 
# It will be stored as a new table called 'city_demographics'
census_final_df.to_sql(name='City_Demographics', con=db_engine, if_exists='append', index=True)

C:\Anaconda\lib\site-packages\pandas\io\sql.py:1144: UserWarning: The provided table name 'City_Demographics' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [34]:
# Confirm tables
db_engine.table_names()

['city_demographics', 'city_health']

In [35]:
# Perform an inner join on the city FIPS codes using MySQL query
# This results in creation of a new aggregated table, named 'city_health_demographics'
db_engine.execute('''create table city_health_demographics
                    select cd.*, ch.*
                    from city_demographics cd
                    inner join city_health ch
                    on cd.city_FIPS = ch.PlaceFIPS;''')